In [192]:
import datetime as dt
import pandas as pd
from pprint import pprint


spx_df = pd.read_csv('SPX_options.csv')
spx_df.head()

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style
0,20201201,20201218,C,100000,3547.6,3570.5,E
1,20201201,20201218,C,200000,3447.6,3470.5,E
2,20201201,20201218,C,300000,3347.7,3370.6,E
3,20201201,20201218,C,400000,3247.7,3270.6,E
4,20201201,20201218,C,500000,3147.7,3170.6,E


In [9]:
spx_df['exdate'].unique()

array([20201218, 20210115, 20210219], dtype=int64)

In [11]:
spx_df['cp_flag'].value_counts()

cp_flag
C    1036
P    1036
Name: count, dtype: int64

In [13]:
today = dt.date(2020, 12, 1)
expiries = [pd.Timestamp(str(x)).date() for x in spx_df['exdate'].unique()]
T = [(exdate-today).days/365.0 for exdate in expiries]
pprint(dict(zip(expiries, T)))

{datetime.date(2020, 12, 18): 0.04657534246575343,
 datetime.date(2021, 1, 15): 0.1232876712328767,
 datetime.date(2021, 2, 19): 0.2191780821917808}


In [15]:
rates_df = pd.read_csv('zero_rates_20201201.csv')
rates_df.head()

,date,days,rate
0,20201201,7,0.102280
1,20201201,13,0.114128
2,20201201,49,0.216480
3,20201201,77,0.220707
4,20201201,104,0.219996


In [17]:
rates_df.tail()

,date,days,rate
40,20201201,3212,0.878441
41,20201201,3303,0.898843
42,20201201,3394,0.918827
43,20201201,3485,0.938031
44,20201201,3576,0.956515


In [19]:
from scipy.interpolate import interp1d

days_to_expiry1 = (expiries[0] - today).days
print(days_to_expiry1)

17


In [21]:
zero_rate_curve = interp1d(rates_df['days'], rates_df['rate'])
rate1 = zero_rate_curve(days_to_expiry1)
print(rate1)

0.12550044444444444


In [27]:
S0 = 3662.45  # Spot price of S&P500 (SPX)
r = rates_df['rate'].iloc[0] / 100  # Risk-free rate, assumed first row
T = 45 / 365  # Time to maturity in years

0.0010228

In [ ]:
# Black-Scholes Model
def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

# Bachelier Model
def BachelierCall(S, K, r, sigma, T):
    d = (S-K) / (S*sigma*np.sqrt(T))
    disc = np.exp(-r*T)
    return disc*((S-K)*norm.cdf(d)+S*sigma*np.sqrt(T)*norm.pdf(d))

def BachelierPut(S, K, r, sigma, T):
    d = (S-K) / (S*sigma*np.sqrt(T))
    disc = np.exp(-r*T)
    return disc*((K-S)*norm.cdf(-d)+S*sigma*np.sqrt(T)*norm.pdf(-d))

In [ ]:
def bachelier_price(S, K, r, sigma, T, option_type='call'):
    d = (S - K) / (S * sigma * np.sqrt(T))
    disc = np.exp(-r * T)
    
    if option_type == 'call':
        return disc * ((S - K) * norm.cdf(d) + S * sigma * np.sqrt(T) * norm.pdf(d))
    elif option_type == 'put':
        return disc * ((K - S) * norm.cdf(-d) + S * sigma * np.sqrt(T) * norm.pdf(-d))
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")


In [30]:
from scipy.stats import norm
import numpy as np

### Payoff function:

$$
\text{Payoff}(S_T) = S_T^{1/3} + 1.5 \times \log(S_T) + 10.0
$$

In [176]:
start = dt.date(2020, 12, 1)
expiry = dt.date(2021, 1, 15)
T = (expiry - start).days/365

x_S = 3662.45
y_S = 366.02

x_K = 3660
y_K = 366

In [257]:
T

0.1232876712328767

In [160]:
def integrand(x):
    return x **  1/3 + 1.5 * np.log(x) + 10.0

In [174]:
r = integrand(T * 365) / 100
r

0.30709993734655483

### 1.Black-Scholes model:

$$
S_T = S_0 \cdot e^{\left(r - \frac{\sigma^2}{2}\right)T + \sigma \sqrt{T} Z}
$$

>the expected value of this payoff

$$
E[V_T]= {S_0}^\frac{1}{3}e^{\frac{rT}{3}}e^{\frac{-\sigma^2T}{18}} + 1.5(log{S_0} + (r-\frac{\sigma^2}{2})T) + 10
$$

>the price:

$$
V_0 = e^{-rT}E[V_T]
$$

In [200]:
x_S = 3662.45
x_K = 3660
r = integrand(T * 365) / 100
T = (expiry - start).days/365
def calculate_atm_volatility(data, S, K, r, T):
    atm_call = data[(data['strike_price'] == K) & (data['cp_flag'] == 'C')]
    atm_put = data[(data['strike_price'] == K) & (data['cp_flag'] == 'P')]

    if not atm_call.empty and not atm_put.empty:
        sigma_call = impliedVolatility(S, K, r, atm_call.iloc[0]['mid_price'], T, 'C')
        sigma_put = impliedVolatility(S, K, r, atm_put.iloc[0]['mid_price'], T, 'P')
        return (sigma_call + sigma_put) / 2
    else:
        return np.nan
spx_bs_sigma = calculate_atm_volatility(spx_df,x_S,x_K,r,T)
spx_bs_sigma

nan

In [194]:
spx_df

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style
0,20201201,20201218,C,100000,3547.6,3570.5,E
1,20201201,20201218,C,200000,3447.6,3470.5,E
2,20201201,20201218,C,300000,3347.7,3370.6,E
3,20201201,20201218,C,400000,3247.7,3270.6,E
4,20201201,20201218,C,500000,3147.7,3170.6,E
...,...,...,...,...,...,...,...
2067,20201201,20210219,P,5000000,1333.1,1350.5,E
2068,20201201,20210219,P,5100000,1431.8,1454.7,E
2069,20201201,20210219,P,5200000,1531.7,1554.6,E
2070,20201201,20210219,P,5300000,1631.5,1654.4,E


In [198]:
# Implied European Options Volatility Model
def impliedCallVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                        BlackScholesCall(S, K, r, x, T),
                        1e-6, 10)
    except Exception:
        impliedVol = np.nan
    return impliedVol

def impliedPutVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                        BlackScholesPut(S, K, r, x, T),
                        1e-6, 10)
    except Exception:
        impliedVol = np.nan
    return impliedVol

def impliedVolatility(S, K, r, price, T, option_type):
    try:
        if option_type == 'C':
            return brentq(lambda x: price - BlackScholesCall(S, K, r, x, T), 1e-12, 10)
        else:
            return brentq(lambda x: price - BlackScholesPut(S, K, r, x, T), 1e-12, 10)
    except Exception:
        return np.nan

def calculate_atm_volatility(spx_data, S, K, r, T):
    atm_call = spx_data[(spx_data['strike_price'] == K) & (spx_data['cp_flag'] == 'C')]
    atm_put = spx_data[(spx_data['strike_price'] == K) & (spx_data['cp_flag'] == 'P')]

    if not atm_call.empty and not atm_put.empty:
        sigma_call = impliedVolatility(S, K, r, atm_call.iloc[0]['mid_price'], T, 'C')
        sigma_put = impliedVolatility(S, K, r, atm_put.iloc[0]['mid_price'], T, 'P')
        return (sigma_call + sigma_put) / 2
    else:
        return np.nan

# calculate_atm_volatility
sigma1 = calculate_atm_volatility(spx_df, S, K, r1, T1)
sigma2 = calculate_atm_volatility(spy_df, S, K, r2, T2)
sigma3 = calculate_atm_volatility(sp, S, K, r3, T3)

sigma=[sigma1 ,sigma2 ,sigma3 ]
sigma

NameError: name 'spx1' is not defined

In [127]:
def black_scholes_price(S, K, r, sigma, T, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

In [129]:
from scipy.optimize import brentq

def impliedVolatility(S, K, r, price, T):
    impliedVol = brentq(lambda x: price -
                        black_scholes_price(S, K, r, x, T),
                        1e-6, 5)

    return impliedVol

In [133]:
import numpy as np

def Black_Scholes_price(S, rate, sigma, T):
    # Term 1: S^(1/3) * exp(rT/3) * exp(-sigma^2T/9)
    term1 = np.power(S, 1.0 / 3.0) * np.exp((rate - 0.5 * sigma**2) * T * (1.0 / 3.0)) * np.exp(0.5 * (1.0 / 9.0) * T * sigma**2)
    
    # Term 2: 1.5 * (log(S) + (rate - 0.5 * sigma^2) * T)
    term2 = 1.5 * (np.log(S) + (rate - 0.5 * sigma**2) * T)
    
    # Term 3: Constant value (10)
    term3 = 10
    
    # Combine all terms and discount to present value
    return np.exp(-rate * T) * (term1 + term2 + term3)


### 2.Bachelier model:

$$S_T = S_0 + \sigma W_T$$

>the expected value of this payoff

$$
E[V_T] = \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} (S_0 + \sigma S_0 \sqrt{T} x)^\frac{1}{3} e^\frac{-x^2}{2}\,dx +
\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} 1.5log(S_0 + \sigma S_0 \sqrt{T} x) e^\frac{-x^2}{2}\,dx
+10
$$

>the price:

$$
V_0 = e^{-rT}E[V_T]
$$

In [87]:
import numpy as np
from scipy.integrate import quad

In [89]:
def integrand_1(x, S0, sigma, T):
    """
    第一部分积分的被积函数:
    (S_0 + sigma * S_0 * sqrt(T) * x)^(1/3) * exp(-x^2 / 2)
    """
    return (
        (S0 + sigma * S0 * np.sqrt(T) * x)**(1/3) 
        * np.exp(-0.5 * x**2)
    )

def integrand_2(x, S0, sigma, T):
    """
    第二部分积分的被积函数:
    1.5 * log(S_0 + sigma * S_0 * sqrt(T) * x) * exp(-x^2 / 2)
    """
    # 确保 log 中的值大于 0
    if S0 + sigma * S0 * np.sqrt(T) * x > 0:
        return (
            1.5 * np.log(S0 + sigma * S0 * np.sqrt(T) * x) 
            * np.exp(-0.5 * x**2)
        )
    else:
        return 0  # 如果 log 的输入为负，则返回 0

def exotic_price(S0, rate, sigma, T):
    """
    使用积分计算 E[V_T] 并返回折现现值
    """
    # 标准正态分布的权重
    normalization = 1 / np.sqrt(2 * np.pi)
    
    # 计算第一个积分
    I1, _ = quad(
        lambda x: integrand_1(x, S0, sigma, T),
        -np.inf, np.inf
    )
    
    # 计算第二个积分
    I2, _ = quad(
        lambda x: integrand_2(x, S0, sigma, T),
        -np.inf, np.inf
    )
    
    # 计算期望值 E[V_T]
    EV_T = normalization * (I1 + I2 + 10)
    
    # 折现到现值
    V0 = np.exp(-rate * T) * EV_T
    
    return V0

### 3. SABR&Static Replication

>The static replication price of the exotic payoff is given by:

$$
\text{Price} = e^{-rT} \cdot h(F) + \int_0^F \phi(K) P(F, K) \, dK + \int_F^\infty \phi(K) C(F, K) \, dK
$$

>Forward Price

$$
   F = S \cdot e^{rT}
$$
   where \( S \) is the current spot price, \( r \) is the risk-free rate, and \( T \) is the time to expiry.

>Exotic Payoff at Forward Price

$$
h(F) = F^{1/3} + 1.5 \cdot \log(F) + 10
$$

>Weight Function

$$
   \phi(K) = -\frac{2}{9} K^{-5/3} - \frac{1.5}{K^2}
$$

In [111]:
import numpy as np
from scipy.stats import norm
from scipy.integrate import quad

>ATM 

$$
\sigma_{\text{imp}} = \frac{\alpha}{F^{1-\beta}} \left[1 + \left(\frac{(1-\beta)^2}{24} \cdot \frac{\alpha^2}{F^{2-2\beta}} + 
0.25 \cdot \rho \cdot \beta \cdot \nu \cdot \frac{\alpha}{F^{1-\beta}} + 
\frac{(2-3\rho^2)}{24} \cdot \nu^2\right) T \right]
$$

>NOT ATM 

$$
\text{numer} =
\alpha \cdot \left[1 + \left(\frac{(1-\beta)^2}{24} \cdot \frac{\alpha^2}{(F K)^{1-\beta}} +
0.25 \cdot \rho \cdot \beta \cdot \nu \cdot \frac{\alpha}{(F K)^{(1-\beta)/2}} +
\frac{(2-3\rho^2)}{24} \cdot \nu^2\right) T \right] \cdot z
$$

where

$$
z = \frac{\nu}{\alpha} \cdot (F K)^{\frac{1 - \beta}{2}} \cdot \ln\left(\frac{F}{K}\right)
$$

and

$$
\chi = \ln\left(\frac{\sqrt{1 - 2\rho z + z^2} + z - \rho}{1 - \rho}\right)
$$

$$
\text{denom} = (F K)^{\frac{1-\beta}{2}} \cdot \left[1 + \frac{(1-\beta)^2}{24} \cdot (\ln(F/K))^2 + 
\frac{(1-\beta)^4}{1920} \cdot (\ln(F/K))^4\right] \cdot \chi
$$

$$
\sigma_{\text{imp}} = \frac{\text{numer}}{\text{denom}}
$$

In [250]:
# SABR implied volatility function
def sabr_volatility(F, K, T, alpha, beta, rho, nu):
    if np.isclose(F, K):  # Handle ATM case
        numer1 = ((1 - beta) ** 2 / 24) * (alpha ** 2) / (F ** (2 - 2 * beta))
        numer2 = 0.25 * rho * beta * nu * alpha / (F ** (1 - beta))
        numer3 = ((2 - 3 * rho ** 2) / 24) * nu ** 2
        return alpha * (1 + (numer1 + numer2 + numer3) * T) / (F ** (1 - beta))
    else:
        z = (nu / alpha) * ((F * K) ** ((1 - beta) / 2)) * np.log(F / K)
        chi = np.log((np.sqrt(1 - 2 * rho * z + z ** 2) + z - rho) / (1 - rho))
        numer = alpha * (1 + (((1 - beta) ** 2 / 24) * (alpha ** 2) / ((F * K) ** (1 - beta)) +
                              0.25 * rho * beta * nu * alpha / ((F * K) ** ((1 - beta) / 2)) +
                              ((2 - 3 * rho ** 2) / 24) * nu ** 2) * T) * z
        denom = ((F * K) ** ((1 - beta) / 2)) * (1 + (1 - beta) ** 2 / 24 * (np.log(F / K)) ** 2 +
                                                 (1 - beta) ** 4 / 1920 * (np.log(F / K)) ** 4) * chi
        return numer / denom


# Black-Scholes call and put pricing
def black_scholes_call(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)


def black_scholes_put(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)


# SABR-based call and put pricing
def sabr_call_price(S, K, r, T, alpha, beta, rho, nu):
    fwd = S * np.exp(r * T)
    sigma = sabr_volatility(fwd, K, T, alpha, beta, rho, nu)
    return black_scholes_call(S, K, r, sigma, T)


def sabr_put_price(S, K, r, T, alpha, beta, rho, nu):
    fwd = S * np.exp(r * T)
    sigma = sabr_volatility(fwd, K, T, alpha, beta, rho, nu)
    return black_scholes_put(S, K, r, sigma, T)


# Integration for static replication
def sabr_call_integrand(K, S, r, T, alpha, beta, rho, nu):
    weight = (-2 / 9) * K ** (-5 / 3) - 1.5 / K ** 2
    return sabr_call_price(S, K, r, T, alpha, beta, rho, nu) * weight


def sabr_put_integrand(K, S, r, T, alpha, beta, rho, nu):
    weight = (-2 / 9) * K ** (-5 / 3) - 1.5 / K ** 2
    return sabr_put_price(S, K, r, T, alpha, beta, rho, nu) * weight


# Static replication price calculation
def static_replication_price(S, r, T, alpha, beta, rho, nu, integration_limits):
    fwd_price = S * np.exp(r * T)
    h_F = (fwd_price ** (1 / 3)) + 1.5 * np.log(fwd_price) + 10

    put_integral = quad(lambda K: sabr_put_integrand(K, S, r, T, alpha, beta, rho, nu), integration_limits[0], fwd_price)[0]
    call_integral = quad(lambda K: sabr_call_integrand(K, S, r, T, alpha, beta, rho, nu), fwd_price, integration_limits[1])[0]

    return np.exp(-r * T) * h_F + put_integral + call_integral


# parameters
spx_params = {"alpha": 1.81600099, "beta": 0.7, "rho": -0.40141522, "nu": 2.79103083}
spy_params = {"alpha": 0.90783421, "beta": 0.7, "rho": -0.48729513, "nu": 2.72885668}
S1 = 3662.45
r = 0.01  
T = 45/365 
integration_limits = (1e-6, 5000)
S2 =366.20
# Calculate static replication prices
spx_price = static_replication_price(S1, r, T, **spx_params, integration_limits=integration_limits)
spy_price = static_replication_price(S2, r, T, **spy_params, integration_limits=integration_limits)

print(f"SPX Static Replication Price: {spx_price:.9f}")
print(f"SPY Static Replication Price: {spy_price:.9f}")

SPX Static Replication Price: 37.669990999
SPY Static Replication Price: 25.972922929


In [265]:
K=3660

In [266]:
weight = (-2 / 9) * K ** (-5 / 3) - 1.5 / K ** 2
weight

-3.676309209604326e-07

### "Model_free" integrated variance:

$$
\sigma_{\text{MF}}^2 T = \mathbb{E} \left[ \int_0^T \sigma_t^2 \, dt \right]
$$

Since 𝜎 is constant under Black-Scholes:

$$
\sigma_{\text{MF}}^2 T = \sigma^2 T
$$

So, integrated variance is simply:

$$
\sigma_{\text{MF}}^2 = \sigma^2
$$

$$
E\Biggl[\int_{0}^{T} \sigma_t^2 \,dt\Biggr] = 2e^{rT}\Biggl(\int_{0}^{F} \frac{P(K)}{K^2} \,dK + \int_{F}^{\infty} \frac{C(K)}{K^2} \,dK\Biggr)
$$

In [253]:
from scipy.integrate import quad
from scipy.stats import norm
import numpy as np

In [ ]:
T = (expiry - start).days/365
spx_S = 3662.45
spy_S = 366.02
r = 0.01
spx_K = 3660
spy_K = 366

In [255]:
# Bachelier Model Integrated Variance Function
def Bach_callintegrand(K, S, r, T, sigma):
    price = BachelierCall(S, K, r, sigma, T) / K**2
    return price

def Bach_putintegrand(K, S, r, T, sigma):
    price = BachelierPut(S, K, r, sigma, T) / K**2
    return price


spx_F = spx_S * np.exp(r * T)
spx_I_put = quad(lambda x: Bach_putintegrand(x, spx_S, r, T, bach_spx_sigma), 0.0, spx_F)
spx_I_call = quad(lambda x: Bach_callintegrand(x, spx_S, r, T, bach_spx_sigma), spx_F, 5000)
spx_Bach_E_var = 2 * np.exp(r * T) * (spx_I_put[0] + spx_I_call[0])
print('The Bachelier expected integrated variance for SPX is: %.9f' % spx_Bach_E_var)


spy_F = spy_S * np.exp(r * T)
spy_I_put = quad(lambda x: Bach_putintegrand(x, spy_S, r, T, bach_spy_sigma), 0.0, spy_F)
spy_I_call = quad(lambda x: Bach_callintegrand(x, spy_S, r, T, bach_spy_sigma), spy_F, 5000)
spy_Bach_E_var = 2 * np.exp(r * T) * (spy_I_put[0] + spy_I_call[0])
print('The Bachelier expected integrated variance for SPY is: %.9f' % spy_Bach_E_var)

The Bachelier expected integrated variance for SPX is: 0.000000000
The Bachelier expected integrated variance for SPY is: 0.000001553


In [ ]:
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S * np.exp(r * T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S * np.exp(r * T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)


def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) * ((- 2 / 9) * K ** (-5 / 3) - 1.5 / (K ** 2))
    return price


def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) * ((- 2 / 9) * K ** (-5 / 3) - 1.5 / (K ** 2))
    return price


#S = 3662.45
#r = interpolated(T * 365) / 100
#T = (expiry - start).days / 365
spx_alpha = 1.81600099
beta = 0.7
spx_rho = -0.40141522
spx_nu = 2.79103083

spy_alpha = 0.90783421
spy_rho = -0.48729513
spy_nu = 2.72885668

spx_F = spx_S * np.exp(r * T)
spy_F = spy_S * np.exp(r * T)

spx_hF = ((spx_S ** (1/3) * np.exp((1/3) * r * T)) + 1.5 * np.log(spx_S * np.exp(r * T)) + 10.0)
spy_hF = ((spy_S ** (1/3) * np.exp((1/3) * r * T)) + 1.5 * np.log(spy_S * np.exp(r * T)) + 10.0)

spx_I_put = quad(lambda x: sabrputintegrand(x, spx_S, r, T, spx_alpha, beta, spx_rho, spx_nu), 1e-6, spx_F)
spx_I_call = quad(lambda x: sabrcallintegrand(x, spx_S, r, T, spx_alpha, beta, spx_rho, spx_nu), spx_F, 5000)
spx_static_replication_price = np.exp(-r * T) * spx_hF + (spx_I_put[0] + spx_I_call[0])

spy_I_put = quad(lambda x: sabrputintegrand(x, spy_S, r, T, spy_alpha, beta, spy_rho, spy_nu), 1e-6, spy_F)
spy_I_call = quad(lambda x: sabrcallintegrand(x, spy_S, r, T, spy_alpha, beta, spy_rho, spy_nu), spy_F, 5000)
spy_static_replication_price = np.exp(-r * T) * spy_hF + (spy_I_put[0] + spy_I_call[0])

print('SPX Static Replication Price: %.9f' % spx_static_replication_price)
print('SPY Static Replication Price: %.9f' % spy_static_replication_price)

In [261]:
spx_alpha = 1.81600099
beta = 0.7
spx_rho = -0.40141522
spx_nu = 2.79103083

spy_alpha = 0.90783421
spy_rho = -0.48729513
spy_nu = 2.72885668

spx_F = spx_S * np.exp(r * T)
spx_I_put = quad(lambda x: sabr_put_integrand(x, spx_S, r, T, spx_alpha, beta, spx_rho, spx_nu), 1e-6, spx_F)
spx_I_call = quad(lambda x: sabr_call_integrand(x, spx_S, r, T, spx_alpha, beta, spx_rho, spx_nu), spx_F, 5000)
spx_E_var = 2 * np.exp(r * T) * (spx_I_put[0] + spx_I_call[0])
print('The static replication expected integrated variance for SPX is: %.9f' % spx_E_var)

spy_F = spy_S * np.exp(r * T)
spy_I_put = quad(lambda x: sabr_put_integrand(x, spy_S, r, T, spy_alpha, beta, spy_rho, spy_nu), 1e-6, spy_F)
spy_I_call = quad(lambda x: sabr_call_integrand(x, spy_S, r, T, spy_alpha, beta, spy_rho, spy_nu), spy_F, 5000)
spy_E_var = 2 * np.exp(r * T) * (spy_I_put[0] + spy_I_call[0])
print('The static replication expected integrated variance for SPY is: %.9f' % spy_E_var)

The static replication expected integrated variance for SPX is: -0.029715832
The static replication expected integrated variance for SPY is: -0.017980583


In [ ]:
spx_F = spx_S * np.exp(r * T)
spx_I_put = quad(lambda x: sabrputintegrand(x, spx_S, r, T, spx_alpha, beta, spx_rho, spx_nu), 1e-6, spx_F)
spx_I_call = quad(lambda x: sabrcallintegrand(x, spx_S, r, T, spx_alpha, beta, spx_rho, spx_nu), spx_F, 5000)
spx_E_var = 2 * np.exp(r * T) * (spx_I_put[0] + spx_I_call[0])
print('The static replication expected integrated variance for SPX is: %.9f' % spx_E_var)

spy_F = spy_S * np.exp(r * T)
spy_I_put = quad(lambda x: sabrputintegrand(x, spy_S, r, T, spy_alpha, beta, spy_rho, spy_nu), 1e-6, spy_F)
spy_I_call = quad(lambda x: sabrcallintegrand(x, spy_S, r, T, spy_alpha, beta, spy_rho, spy_nu), spy_F, 5000)
spy_E_var = 2 * np.exp(r * T) * (spy_I_put[0] + spy_I_call[0])
print('The static replication expected integrated variance for SPY is: %.9f' % spy_E_var)

In [269]:
def static_replication_price2(S, r, T, alpha, beta, rho, nu, integration_limits):
    fwd_price = S * np.exp(r * T)
    h_F = (fwd_price ** (1 / 3)) + 1.5 * np.log(fwd_price) + 10

    put_integral = quad(lambda K: sabr_put_integrand(K, S, r, T, alpha, beta, rho, nu), integration_limits[0], fwd_price)[0]
    call_integral = quad(lambda K: sabr_call_integrand(K, S, r, T, alpha, beta, rho, nu), fwd_price, integration_limits[1])[0]

    return 2 * np.exp(r * T) *(put_integral + call_integral)


# parameters
spx_params = {"alpha": 1.81600099, "beta": 0.7, "rho": -0.40141522, "nu": 2.79103083}
spy_params = {"alpha": 0.90783421, "beta": 0.7, "rho": -0.48729513, "nu": 2.72885668}
S1 = 3662.45
r = 0.01  
T = 45/365 
integration_limits = (1e-6, 5000)
S2 =366.20
# Calculate static replication prices
spx_price2 = static_replication_price2(S1, r, T, **spx_params, integration_limits=integration_limits)
spy_price2 = static_replication_price2(S2, r, T, **spy_params, integration_limits=integration_limits)

print(spx_price2)
print(spy_price2)

-0.029715832436617892
-0.017976458627835555
